<a href="https://colab.research.google.com/github/RinayGajjar/NewsHeadLineGenerator/blob/main/HeadLineGenerator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf

In [3]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, SimpleRNN
from tensorflow.keras.layers import LSTM

In [ ]:
df = pd.read_csv('news_summary_more.csv')

In [6]:
df

,headlines,text
0,upGrad learner switches to career in ML & Al w...,"Saurav Kant, an alumnus of upGrad and IIIT-B's..."
1,Delhi techie wins free food from Swiggy for on...,Kunal Shah's credit card bill payment platform...
2,New Zealand end Rohit Sharma-led India's 12-ma...,New Zealand defeated India by 8 wickets in the...
3,Aegon life iTerm insurance plan helps customer...,"With Aegon Life iTerm Insurance plan, customer..."
4,"Have known Hirani for yrs, what if MeToo claim...",Speaking about the sexual harassment allegatio...
...,...,...
98396,CRPF jawan axed to death by Maoists in Chhatti...,A CRPF jawan was on Tuesday axed to death with...
98397,First song from Sonakshi Sinha's 'Noor' titled...,"'Uff Yeh', the first song from the Sonakshi Si..."
98398,'The Matrix' film to get a reboot: Reports,"According to reports, a new version of the 199..."
98399,Snoop Dogg aims gun at clown dressed as Trump ...,A new music video shows rapper Snoop Dogg aimi...


In [7]:
df.columns

Index(['headlines', 'text'], dtype='object')

In [8]:
token_df = Tokenizer()

In [9]:
token_df.fit_on_texts(df['headlines']+df['text'])##tokenized headlines and text data
vocab_size = len(token_df.word_index)+1

In [10]:
##converting into sequences
text_seq = token_df.texts_to_sequences(df['text'])
headlines_seq = token_df.texts_to_sequences(df['headlines'])

In [11]:
max_text_len = max(len(text) for text in text_seq)
max_headlines_len = max(len(headlines) for headlines in headlines_seq)
text_padded = pad_sequences(text_seq, maxlen=max_text_len, padding='pre')
headlines_padded = pad_sequences(headlines_seq, maxlen=max_headlines_len, padding='pre')

In [12]:
##splitting into input and output vars
X = text_padded
y = headlines_padded

In [13]:
X

array([[     0,      0,      0, ...,    110,    101,  10436],
       [     0,      0,      0, ...,   2732,      6,    116],
       [     0,      0,      0, ...,    385,    198,    699],
       ...,
       [     0,      0,      0, ...,     14,     56,  29051],
       [     0,      0,      0, ...,      1,    507, 163370],
       [     0,      0,      0, ...,      2,      1,     55]], dtype=int32)

In [14]:
y

array([[    0,     0,     0, ...,  1321,  1506,  2112],
       [    0,     0,     0, ...,    30,     7, 29128],
       [    0,     0,     0, ...,   162,   675,  5529],
       ...,
       [    0,     0,     0, ...,     5, 12127,    94],
       [    0,     0,     0, ...,     3,    65,   117],
       [    0,     0,     0, ...,     2,  9341,    55]], dtype=int32)

In [15]:
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=50, input_length=max_text_len))
model.add(LSTM(units=128, return_sequences=False))
model.add(Dense(units=32, activation='relu'))
model.add(Dense(units=vocab_size, activation='softmax'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [16]:
model.compile(optimizer='adam', loss= 'categorical_crossentropy', metrics=['accuracy'])

In [17]:
model.build(input_shape=(None, max_text_len))

In [18]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 77, 50)              │       8,168,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn (SimpleRNN)               │ (None, 77, 64)              │           7,360 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 77, 32)              │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 77, 163372)          │       5,391,276 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 13,569,316 (51.76 MB)

 Trainable params: 13,569,316 (51.76 MB)

 Non-trainable params: 0 (0.00 B)

In [19]:
from tensorflow.keras.utils import to_categorical

# Shift y to predict the next token
y = headlines_padded[:, 1:]

# One-hot encode the target sequences
y = to_categorical(y, num_classes=vocab_size)

In [ ]:
model.fit(X,y,epochs=10,batch_size=1,verbose=1)